In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import broadcast, col

spark = SparkSession.builder.appName("Broadcast Join").getOrCreate()

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

df_medals = spark.read.option("header", "true").csv("/home/iceberg/data/medals.csv")
df_medals_matches_players = spark.read.option("header", "true").csv("/home/iceberg/data/medals_matches_players.csv")
df_matches = spark.read.option("header", "true").csv("/home/iceberg/data/matches.csv")
df_maps = spark.read.option("header", "true").csv("/home/iceberg/data/maps.csv")

df_medals_join_match_id = df_medals_matches_players.join(df_medals, on="medal_id", how="inner")
df_match_join_map_id = df_medals_join_match_id.join(df_matches, on="match_id", how="inner")
df_medals_maps = df_match_join_map_id.join(broadcast(df_maps), on="mapid", how="inner")
df_medals_maps.show()

25/07/02 10:51:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/07/02 10:51:04 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
[Stage 10:==========================>                              (6 + 7) / 13]

+--------------------+--------------------+----------+---------------+-----+--------------------+-----------+----------+------------------+-------------------+------------+-------------+-----------------+--------------------+-------------+----------+------------+--------------------+--------------------+-------------+--------------------+--------------+---------+--------------+------+--------------------+
|               mapid|            match_id|  medal_id|player_gamertag|count|          sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|   classification|         description|         name|difficulty|is_team_game|         playlist_id|     game_variant_id|is_match_over|     completion_date|match_duration|game_mode|map_variant_id|  name|         description|
+--------------------+--------------------+----------+---------------+-----+--------------------+-----------+----------+------------------+-------------------+------------+----------

In [1]:
spark.sql("""CREATE DATABASE IF NOT EXISTS bootcamp""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.match_details_bucketed""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
spark.sql("""DROP TABLE IF EXISTS bootcamp.medal_matches_players_bucketed""")

# Load match details from csv
df_match_details = spark.read.option("header", "true").csv("/home/iceberg/data/match_details.csv")
# DDL for match details
match_details_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
    match_id STRING,
    player_gamertag STRING,
    total_xp INT,
    player_rank_on_team INT,
    player_finished BOOLEAN,
    player_total_kills INT,
    player_total_deaths INT,
    player_total_assists INT,
    did_win INT,
    team_id INT
) USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""
spark.sql(match_details_ddl)
df_match_details.select(
    "match_id",
    "player_gamertag",
    "total_xp",
    "player_rank_on_team",
    "player_finished",
    "player_total_kills",
    "player_total_deaths",
    "player_total_assists",
    "did_win",
    "team_id"
).write.mode("overwrite").bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

# DDL for matches
matches_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
    match_id STRING,
    mapid STRING,
    is_team_game BOOLEAN,
    playlist_id STRING,
    game_variant_id STRING,
    game_mode STRING,
    map_variant_id STRING
) USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""
spark.sql(matches_ddl)
df_matches.select(
    "match_id",
    "mapid",
    "is_team_game",
    "playlist_id",
    "game_variant_id",
    "game_mode",
    "map_variant_id"
).write.mode("overwrite").bucketBy(16, "match_id").saveAsTable(
    "bootcamp.matches_bucketed"
)

# DDL for medals, matches and players
medal_matches_players_ddl = """
CREATE TABLE IF NOT EXISTS bootcamp.medal_matches_players_bucketed (
    match_id STRING,
    player_gamertag STRING,
    medal_id BIGINT,
    count INT
) USING iceberg
PARTITIONED BY (bucket(16, match_id));
"""
spark.sql(medal_matches_players_ddl)
df_medals_matches_players.select(
    "match_id",
    "player_gamertag",
    "medal_id",
    "count"
).write.mode("overwrite").bucketBy(16, "match_id").saveAsTable(
    "bootcamp.medal_matches_players_bucketed"
)

bucket_result = spark.sql("""
    SELECT
        mdb.match_id AS match_id,
        mdb.player_gamertag AS player_gamertag,
        mdb.total_xp AS total_xp,
        mdb.player_rank_on_team AS player_rank_on_team,
        mdb.player_finished AS player_finished,
        mdb.player_total_kills AS player_total_kills,
        mdb.player_total_deaths AS player_total_deaths,
        mdb.player_total_assists AS player_total_assists,
        mdb.did_win AS did_win,
        mdb.team_id AS team_id,
        md.mapid AS mapid,
        md.is_team_game AS is_team_game,
        md.playlist_id AS playlist_id,
        md.game_variant_id AS game_variant_id,
        md.game_mode AS game_mode,
        md.map_variant_id AS map_variant_id,
        mmp.medal_id AS medal_id,
        mmp.count AS count
    FROM bootcamp.match_details_bucketed mdb 
    JOIN bootcamp.matches_bucketed md ON mdb.match_id = md.match_id
    JOIN bootcamp.medal_matches_players_bucketed mmp ON mdb.match_id = mmp.match_id
""")

Py4JJavaError: An error occurred while calling o36.sql.
: org.apache.iceberg.exceptions.ServiceFailureException: Server error: NotFoundException: Location does not exist: s3://warehouse/bootcamp/match_details_bucketed/metadata/00001-8cf37a8c-f298-4478-97b0-b8f76e52fc53.metadata.json
	at org.apache.iceberg.rest.ErrorHandlers$DefaultErrorHandler.accept(ErrorHandlers.java:217)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:118)
	at org.apache.iceberg.rest.ErrorHandlers$TableErrorHandler.accept(ErrorHandlers.java:102)
	at org.apache.iceberg.rest.HTTPClient.throwFailure(HTTPClient.java:224)
	at org.apache.iceberg.rest.HTTPClient.execute(HTTPClient.java:308)
	at org.apache.iceberg.rest.BaseHTTPClient.get(BaseHTTPClient.java:77)
	at org.apache.iceberg.rest.RESTClient.get(RESTClient.java:97)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadInternal(RESTSessionCatalog.java:465)
	at org.apache.iceberg.rest.RESTSessionCatalog.loadTable(RESTSessionCatalog.j

In [3]:
bucket_result.explain()

NameError: name 'bucket_result' is not defined

In [4]:
bucket_result.createOrReplaceTempView("bucket_result")

spark.sql("""
    SELECT * FROM bucket_result
""").show()

[Stage 28:=============================>                            (2 + 2) / 4]

+--------------------+---------------+--------+-------------------+---------------+------------------+-------------------+--------------------+-------+-------+--------------------+------------+--------------------+--------------------+---------+--------------+----------+-----+
|            match_id|player_gamertag|total_xp|player_rank_on_team|player_finished|player_total_kills|player_total_deaths|player_total_assists|did_win|team_id|               mapid|is_team_game|         playlist_id|     game_variant_id|game_mode|map_variant_id|  medal_id|count|
+--------------------+---------------+--------+-------------------+---------------+------------------+-------------------+--------------------+-------+-------+--------------------+------------+--------------------+--------------------+---------+--------------+----------+-----+
|00169217-cca6-4b4...|  King Terror V| 1601153|                  1|          false|                14|                  7|                   2|      1|      1|cc040aa

In [27]:
# - Which player averages the most kills per game?
spark.sql("""
    SELECT
        player_gamertag,
        COUNT(1) AS num_
        AVG(player_total_kills) AS avg_kills_per_game
    FROM bucket_result
    GROUP BY player_gamertag
    ORDER BY avg_kills_per_game DESC
""").show()

+---------------+------------------+
|player_gamertag|avg_kills_per_game|
+---------------+------------------+
|   gimpinator14|             109.0|
|  I Johann117 I|              96.0|
|BudgetLegendary|              83.0|
|      GsFurreal|              75.0|
|   TameablePoet| 74.22429906542057|
|   Sexy is Back|              73.0|
|   killerguy789|              68.0|
|THC GUILTYSPARK|              67.0|
|    HisLattice1|              66.0|
|PrimePromethean|              66.0|
|     Profit TKO| 64.38181818181818|
|     taurenmonk|              64.0|
|  DBossCnDTEXAS| 63.47586206896552|
|   Dinosaur B0B|              63.0|
|WhiteMountainDC|              63.0|
|        Darugis|              62.0|
|     MONKEYBAKE|              62.0|
|       BlightNB|              62.0|
|    ManicZ0mb1e|              61.0|
|Lord Leonidamir|              60.0|
+---------------+------------------+
only showing top 20 rows



In [6]:
# - Which playlist gets played the most?
spark.sql("""
    SELECT
        playlist_id,
        COUNT(1) AS count
    FROM bucket_result
    GROUP BY playlist_id
    ORDER BY count DESC
""").show()

+--------------------+-------+
|         playlist_id|  count|
+--------------------+-------+
|f72e0ef0-7c4a-430...|1565529|
|780cc101-005c-4fc...|1116002|
|0bcf2be1-3168-4e4...|1015496|
|c98949ae-60a8-43d...| 824932|
|2323b76a-db98-4e0...| 692342|
|892189e9-d712-4bd...| 667670|
|f27a65eb-2d11-496...| 167498|
|355dc154-9809-4ed...| 140006|
|d0766624-dbd7-453...| 138470|
|bc0f8ad6-31e6-4a1...| 111073|
|7b7e892c-d9b7-4b0...|  82723|
|7385b4a1-86bf-4ae...|  76425|
|f0c9ef9a-48bd-4b2...|  47813|
|b5d5a242-ffa5-4d8...|  46411|
|819eb188-1a1c-48b...|  39404|
|d21c8381-26f1-4d6...|  37049|
|4b12472e-2a06-423...|  28733|
|5728f612-3f20-445...|  28543|
|0504ca3c-de41-48f...|  22502|
|88b7de19-113c-4be...|  15860|
+--------------------+-------+
only showing top 20 rows



In [7]:
# - Which map gets played the most?
spark.sql("""
    SELECT
        mapid,
        COUNT(1) AS plays_count
    FROM bucket_result
    GROUP BY mapid
    ORDER BY plays_count DESC
""").show()

+--------------------+-----------+
|               mapid|plays_count|
+--------------------+-----------+
|c74c9d0f-f206-11e...|    1445545|
|c7edbf0f-f206-11e...|    1435048|
|c7805740-f206-11e...|     953278|
|cdb934b0-f206-11e...|     396305|
|cb914b9e-f206-11e...|     309045|
|ce1dc2de-f206-11e...|     299736|
|cebd854f-f206-11e...|     298891|
|caacb800-f206-11e...|     291540|
|cd844200-f206-11e...|     261162|
|cc040aa1-f206-11e...|     256966|
|cdee4e70-f206-11e...|     244295|
|c7b7baf0-f206-11e...|     204568|
|ca737f8f-f206-11e...|     156631|
|cbcea2c0-f206-11e...|     135375|
|cc74f4e1-f206-11e...|     132392|
|ce89a40f-f206-11e...|      65081|
+--------------------+-----------+



In [8]:
# - Which map do players get the most Killing Spree medals on?
spark.sql("""
    SELECT
        mapid,
        SUM(count) AS spree_count
    FROM bucket_result
    WHERE medal_id = 2430242797
    GROUP BY mapid
    ORDER BY spree_count DESC
""").show()

25/07/01 13:30:31 WARN DataSourceV2Strategy: Can't translate true to source filter, unsupported expression


+--------------------+-----------+
|               mapid|spree_count|
+--------------------+-----------+
|c74c9d0f-f206-11e...|    71863.0|
|c7edbf0f-f206-11e...|    51973.0|
|c7805740-f206-11e...|    46833.0|
|cdb934b0-f206-11e...|    16860.0|
|ce1dc2de-f206-11e...|    15509.0|
|cb914b9e-f206-11e...|    14962.0|
|cebd854f-f206-11e...|    14777.0|
|caacb800-f206-11e...|    14380.0|
|cd844200-f206-11e...|    13382.0|
|cdee4e70-f206-11e...|    13208.0|
|cc040aa1-f206-11e...|    12579.0|
|c7b7baf0-f206-11e...|    10076.0|
|ca737f8f-f206-11e...|     7691.0|
|cc74f4e1-f206-11e...|     7138.0|
|cbcea2c0-f206-11e...|     6899.0|
|ce89a40f-f206-11e...|     2640.0|
+--------------------+-----------+



In [24]:
# Task 5 - With the aggregated data set; try different `.sortWithinPartitions` to see which has the smallest data size (hint: playlists and maps are both very low cardinality)
sorted_bucket_result = bucket_result.repartition(10, col("match_id")).sortWithinPartitions(col("playlist_id"), col("mapid"), col("game_variant_id"), col("team_id"))
sorted_bucket_result.write.mode("overwrite").saveAsTable("bootcamp.sorted_bucket_result")
spark.sql("""
    SELECT
        SUM(file_size_in_bytes) AS size,
        COUNT(1) AS num_files
    FROM bootcamp.sorted_bucket_result.files
""").show()

+--------+---------+
|    size|num_files|
+--------+---------+
|15561528|       10|
+--------+---------+



In [10]:
15561528 - col("playlist_id"), col("mapid"), col("game_variant_id"), col("team_id")
15636964 - col("playlist_id"), col("mapid"), col("team_id"), col("game_variant_id")
15688851 - col("playlist_id"), col("mapid")
15817563 - col("mapid"), col("playlist_id"), col("game_variant_id")
15876151 - col("mapid"), col("game_variant_id"), col("playlist_id")
15881785 - col("game_variant_id"), col("playlist_id"), col("mapid")
16042944 - col("team_id"), col("game_variant_id"), col("playlist_id"), col("mapid")

(Column<'(15688851 - playlist_id)'>, Column<'mapid'>)

In [21]:
spark.sql("""
    SELECT
        COUNT(DISTINCT team_id) AS team_id,
        COUNT(DISTINCT mapid) AS mapid,
        COUNT(DISTINCT playlist_id) AS playlist_id,
        COUNT(DISTINCT game_variant_id) AS game_variant_id,
        COUNT(DISTINCT map_variant_id) AS map_variant_id,
        COUNT(DISTINCT medal_id) AS medal_id
    FROM bucket_result
""").show()

[Stage 217:>                                                      (0 + 10) / 10]

+-------+-----+-----------+---------------+--------------+--------+
|team_id|mapid|playlist_id|game_variant_id|map_variant_id|medal_id|
+-------+-----+-----------+---------------+--------------+--------+
|      8|   16|         23|              9|           584|     142|
+-------+-----+-----------+---------------+--------------+--------+

